# 🎯 NetGuardian-AI : Approche Hybride

**Objectif** : Implémenter un système de détection hybride avec 2 modèles en cascade

- **Modèle 1** : Détection binaire (Normal vs Attaque)
- **Modèle 2** : Classification multi-classes (Type d'attaque)

---

## 📦 Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
import joblib
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

ModuleNotFoundError: No module named 'pandas'

## 📂 Charger le Dataset

In [ ]:
# Charger le dataset pré-nettoyé
df = pd.read_csv('/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv')

print(f"Shape: {df.shape}")
print(f"\nDistribution des classes:")
print(df['Attack Type'].value_counts())

## 🧹 Nettoyage Rapide

In [ ]:
# Remplacer infinis et NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remplir NaN avec médiane
for col in df.select_dtypes(include=[np.number]).columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# Supprimer duplications
df = df.drop_duplicates()

print(f"✅ Dataset nettoyé: {df.shape}")

## 🏷️ Créer les 2 Types de Labels

In [ ]:
# 1. Label Binaire : 0 = Normal, 1 = Attaque
df['Binary_Label'] = (df['Attack Type'] != 'Normal Traffic').astype(int)

print("Distribution binaire:")
print(df['Binary_Label'].value_counts())
print(f"\nPourcentages:")
print(df['Binary_Label'].value_counts(normalize=True) * 100)

In [ ]:
# 2. Label Multi-Classes : Fusionner DoS et DDoS
df['Attack_Merged'] = df['Attack Type'].replace({
    'DoS': 'DoS_DDoS',
    'DDoS': 'DoS_DDoS'
})

print("Distribution après fusion:")
print(df['Attack_Merged'].value_counts())

In [ ]:
# Encoder les labels multi-classes
le = LabelEncoder()
df['Multiclass_Label'] = le.fit_transform(df['Attack_Merged'])

# Sauvegarder l'encodeur
joblib.dump(le, 'label_encoder.pkl')

print("Mapping numérique:")
for i, label in enumerate(le.classes_):
    count = (df['Multiclass_Label'] == i).sum()
    print(f"{i}: {label:20s} {count:8,} instances")

## 📊 Visualisation de la Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Binaire
df['Binary_Label'].value_counts().plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Distribution Binaire', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Label (0=Normal, 1=Attaque)')
axes[0].set_ylabel('Nombre d\'Instances')
axes[0].set_xticklabels(['Normal', 'Attaque'], rotation=0)

# Multi-classes
df['Attack_Merged'].value_counts().plot(kind='bar', ax=axes[1], color='skyblue')
axes[1].set_title('Distribution Multi-Classes', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Type d\'Attaque')
axes[1].set_ylabel('Nombre d\'Instances')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('distribution_labels.png', dpi=300, bbox_inches='tight')
plt.show()

## 🎯 Préparation des Features

In [ ]:
# Séparer features et labels
label_cols = ['Attack Type', 'Binary_Label', 'Attack_Merged', 'Multiclass_Label']
feature_cols = [col for col in df.columns if col not in label_cols]

X = df[feature_cols]
y_binary = df['Binary_Label']
y_multiclass = df['Multiclass_Label']

print(f"Features: {X.shape}")
print(f"Binary labels: {y_binary.shape}")
print(f"Multiclass labels: {y_multiclass.shape}")

## 🔀 Split Train/Test

In [ ]:
# Split stratifié pour garder la distribution
X_train, X_test, y_binary_train, y_binary_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# Récupérer aussi les labels multi-classes correspondants
y_multi_train = y_multiclass.loc[X_train.index]
y_multi_test = y_multiclass.loc[X_test.index]

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")
print(f"\nDistribution train (binaire):")
print(y_binary_train.value_counts())

## 📏 Normalisation

In [ ]:
# Normaliser les features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Sauvegarder le scaler
joblib.dump(scaler, 'scaler.pkl')

print(f"✅ Features normalisées")
print(f"Mean: {X_train_scaled.mean():.6f}")
print(f"Std: {X_train_scaled.std():.6f}")

## 🎯 MODÈLE 1 : Détection Binaire

**Objectif** : Détecter si le trafic est normal ou une attaque

In [ ]:
# Calculer le poids pour équilibrer
scale_pos_weight = len(y_binary_train[y_binary_train==0]) / len(y_binary_train[y_binary_train==1])
print(f"Scale pos weight: {scale_pos_weight:.2f}")

In [ ]:
# Entraîner XGBoost pour détection binaire
print("🚀 Entraînement du Modèle 1 (Détection Binaire)...")

model1 = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

model1.fit(X_train_scaled, y_binary_train)
print("✅ Modèle 1 entraîné")

In [ ]:
# Prédictions
y_binary_pred = model1.predict(X_test_scaled)
y_binary_proba = model1.predict_proba(X_test_scaled)[:, 1]

# Évaluation
print("="*70)
print("MODÈLE 1 : DÉTECTION BINAIRE")
print("="*70)
print(classification_report(y_binary_test, y_binary_pred, target_names=['Normal', 'Attaque']))

# AUC-ROC
auc = roc_auc_score(y_binary_test, y_binary_proba)
print(f"\nAUC-ROC: {auc:.4f}")

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_binary_test, y_binary_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Attaque'],
            yticklabels=['Normal', 'Attaque'])
plt.title('Matrice de Confusion - Modèle 1 (Binaire)', fontsize=14, fontweight='bold')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.tight_layout()
plt.savefig('confusion_matrix_binary.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Sauvegarder le modèle 1
joblib.dump(model1, 'model1_binary.pkl')
print("✅ Modèle 1 sauvegardé: model1_binary.pkl")

## 🎯 MODÈLE 2 : Classification Multi-Classes

**Objectif** : Identifier le type d'attaque exact

**Stratégie** : Entraîner seulement sur les attaques détectées

In [ ]:
# Filtrer seulement les attaques dans le train set
attack_mask_train = y_binary_train == 1
X_train_attacks = X_train_scaled[attack_mask_train]
y_multi_train_attacks = y_multi_train[attack_mask_train]

print(f"Données d'entraînement (attaques seulement): {X_train_attacks.shape}")
print(f"\nDistribution:")
print(y_multi_train_attacks.value_counts())

In [ ]:
# Appliquer SMOTE pour équilibrer les classes minoritaires
print("🔄 Application de SMOTE...")

smote = SMOTE(random_state=42, k_neighbors=5)
X_train_resampled, y_multi_resampled = smote.fit_resample(X_train_attacks, y_multi_train_attacks)

print(f"✅ Après SMOTE: {X_train_resampled.shape}")
print(f"\nDistribution après SMOTE:")
print(pd.Series(y_multi_resampled).value_counts())

In [ ]:
# Entraîner XGBoost pour classification multi-classes
print("🚀 Entraînement du Modèle 2 (Multi-Classes)...")

model2 = XGBClassifier(
    n_estimators=300,
    max_depth=12,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

model2.fit(X_train_resampled, y_multi_resampled)
print("✅ Modèle 2 entraîné")

In [ ]:
# Évaluer sur les attaques du test set
attack_mask_test = y_binary_test == 1
X_test_attacks = X_test_scaled[attack_mask_test]
y_multi_test_attacks = y_multi_test[attack_mask_test]

y_multi_pred = model2.predict(X_test_attacks)

print("="*70)
print("MODÈLE 2 : CLASSIFICATION MULTI-CLASSES")
print("="*70)
print(classification_report(y_multi_test_attacks, y_multi_pred, target_names=le.classes_))

In [ ]:
# Matrice de confusion multi-classes
cm_multi = confusion_matrix(y_multi_test_attacks, y_multi_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_multi, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.title('Matrice de Confusion - Modèle 2 (Multi-Classes)', fontsize=14, fontweight='bold')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix_multiclass.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Sauvegarder le modèle 2
joblib.dump(model2, 'model2_multiclass.pkl')
print("✅ Modèle 2 sauvegardé: model2_multiclass.pkl")

## 🔗 Système Hybride : Combinaison des 2 Modèles

In [ ]:
class HybridIDS:
    """
    Système de détection d'intrusion hybride
    Combine détection binaire et classification multi-classes
    """
    
    def __init__(self, binary_model, multiclass_model, scaler, label_encoder):
        self.binary_model = binary_model
        self.multiclass_model = multiclass_model
        self.scaler = scaler
        self.le = label_encoder
    
    def predict(self, X):
        """
        Prédiction hybride en 2 étapes
        
        Args:
            X: Features (DataFrame ou array)
        
        Returns:
            Liste de dictionnaires avec type et confiance
        """
        # Normaliser
        X_scaled = self.scaler.transform(X)
        
        # Étape 1 : Détection binaire
        is_attack = self.binary_model.predict(X_scaled)
        binary_proba = self.binary_model.predict_proba(X_scaled)
        
        results = []
        
        for i, (attack_flag, proba) in enumerate(zip(is_attack, binary_proba)):
            if attack_flag == 0:
                # Trafic normal
                results.append({
                    'type': 'Normal Traffic',
                    'confidence': float(proba[0]),
                    'is_attack': False
                })
            else:
                # Attaque détectée → Étape 2 : Classification
                attack_type_encoded = self.multiclass_model.predict(X_scaled[i:i+1])[0]
                attack_proba = self.multiclass_model.predict_proba(X_scaled[i:i+1])[0]
                attack_type = self.le.inverse_transform([attack_type_encoded])[0]
                
                results.append({
                    'type': attack_type,
                    'confidence': float(attack_proba.max()),
                    'is_attack': True
                })
        
        return results
    
    def predict_df(self, X):
        """
        Prédiction avec résultat en DataFrame
        """
        results = self.predict(X)
        return pd.DataFrame(results)

print("✅ Classe HybridIDS créée")

In [ ]:
# Créer le système hybride
hybrid_ids = HybridIDS(
    binary_model=model1,
    multiclass_model=model2,
    scaler=scaler,
    label_encoder=le
)

print("✅ Système hybride initialisé")

In [ ]:
# Tester sur un échantillon
sample_size = 100
X_sample = X_test.iloc[:sample_size]
y_true = df.loc[X_sample.index, 'Attack Type']

# Prédictions hybrides
predictions = hybrid_ids.predict_df(X_sample)

# Comparer avec la vérité
comparison = pd.DataFrame({
    'True_Label': y_true.values,
    'Predicted_Label': predictions['type'].values,
    'Confidence': predictions['confidence'].values,
    'Is_Attack': predictions['is_attack'].values
})

print("Échantillon de prédictions:")
print(comparison.head(20))

In [ ]:
# Calculer l'accuracy du système hybride
correct = (comparison['True_Label'] == comparison['Predicted_Label']).sum()
accuracy = correct / len(comparison) * 100

print(f"\n{'='*70}")
print(f"PERFORMANCE DU SYSTÈME HYBRIDE")
print(f"{'='*70}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Prédictions correctes: {correct}/{len(comparison)}")
print(f"{'='*70}")

## 💾 Sauvegarder le Système Complet

In [ ]:
# Sauvegarder tous les composants
import pickle

hybrid_system = {
    'binary_model': model1,
    'multiclass_model': model2,
    'scaler': scaler,
    'label_encoder': le,
    'feature_names': X.columns.tolist()
}

with open('hybrid_ids_system.pkl', 'wb') as f:
    pickle.dump(hybrid_system, f)

print("✅ Système hybride complet sauvegardé: hybrid_ids_system.pkl")

## 📤 Export vers Google Drive

In [ ]:
# 💾 Sauvegarder sur Google Drive
import os
import shutil

def save_to_drive(source_file, destination_folder='NetGuardian_Models/Hybrid'):
    """Sauvegarde vers Google Drive"""
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
        
        drive_path = f"/content/drive/MyDrive/{destination_folder}"
        os.makedirs(drive_path, exist_ok=True)
        
        if os.path.exists(source_file):
            dest = os.path.join(drive_path, os.path.basename(source_file))
            shutil.copy2(source_file, dest)
            print(f"✅ Sauvegardé sur Drive: {dest}")
        else:
            print(f"⚠️ Fichier introuvable: {source_file}")
        
    except ImportError:
        print("⚠️ Pas sur Colab. Sauvegarde locale conservée.")
    except Exception as e:
        print(f"❌ Erreur: {e}")

# Exporter les fichiers clés
save_to_drive('hybrid_ids_system.pkl')
save_to_drive('scaler.pkl')
save_to_drive('label_encoder.pkl')
save_to_drive('model1_binary.pkl')
save_to_drive('model2_multiclass.pkl')

print("\n🚀 Export terminé !")

## 📊 Résumé Final

In [ ]:
print("="*70)
print("RÉSUMÉ DE L'ENTRAÎNEMENT HYBRIDE")
print("="*70)
print(f"\n📊 Dataset:")
print(f"   - Total instances: {len(df):,}")
print(f"   - Features: {X.shape[1]}")
print(f"   - Classes: {len(le.classes_)}")

print(f"\n🎯 Modèle 1 (Binaire):")
print(f"   - Type: XGBoost")
print(f"   - Classes: Normal vs Attaque")
print(f"   - Fichier: model1_binary.pkl")

print(f"\n🎯 Modèle 2 (Multi-Classes):")
print(f"   - Type: XGBoost + SMOTE")
print(f"   - Classes: {len(le.classes_)}")
print(f"   - Fichier: model2_multiclass.pkl")

print(f"\n💾 Fichiers générés:")
print(f"   - model1_binary.pkl")
print(f"   - model2_multiclass.pkl")
print(f"   - scaler.pkl")
print(f"   - label_encoder.pkl")
print(f"   - hybrid_ids_system.pkl")

print(f"\n✅ Système hybride prêt pour déploiement!")
print("="*70)

## 📥 Prochaines Étapes

1. ✅ Système hybride entraîné et testé
2. ✅ Modèles sauvegardés sur Drive
3. ⏳ Intégrer dans NetGuardian-AI dashboard
4. ⏳ Tester en temps réel

---

**Félicitations ! Votre IDS hybride est prêt ! 🎉**